In [ ]:
from utils import *
import getpass

# from landsatxplore.earthexplorer import EarthExplorer
import tarfile
import boto3

keep_original_band_scenes = False
one_per_month = True
dir_suffix = ""
if (dir_suffix != "") and (not dir_suffix.endswith("/")):
    dir_suffix = dir_suffix + "/"

aws_session = rasterio.session.AWSSession(boto3.Session(), requester_pays=True)
# aws_session = rasterio.session.AWSSession(boto3.Session())

In [ ]:
wa_bbox = resize_bbox(
    read_kml_polygon("data/inputs_old/LANDSAT_8_127111/WA.kml")[1], 0.1
)
aoi_polys = kml_to_poly("data/inputs_old/aois.kml").geoms

bbox_list = [
    [67.45, -72.55, 67.55, -72.45],  # Amery bed rock
    [69.2, -68.1, 69.4, -67.9],  # Amery top
    [wa_bbox.left, wa_bbox.bottom, wa_bbox.right, wa_bbox.top],  # WA sand dunes
    *[list(p.bounds) for p in aoi_polys],  # AOI polygons
]

In [ ]:
platform = "LANDSAT_7"
r_channel = "red"
g_channel = "green"
b_channel = "blue"
query = get_search_query(
    bbox_list[0],
    start_date="",
    platform=platform,
)  # Amery ice shelf bed rock

server_url = "https://landsatlook.usgs.gov/stac-server/search"
features = query_stac_server(query, server_url)

In [ ]:
scene_dict, scene_list = find_scenes_dict(
    features,
    one_per_month=True,
    start_end_years=[2009, 2010],
    acceptance_list=["red", "green", "blue"],
)
path_rows = list(scene_dict.keys())
print(path_rows)
dates = [list(scene_dict[pr].keys()) for pr in path_rows]
print(dates)

In [ ]:
# sn = combine_scene_dicts([scene_dict['126111'], scene_dict['127111']])
# sn_closest_pair = get_pair_dict(sn, "closest")
# sn_farthest_pair = get_pair_dict(sn, "farthest")

In [ ]:
path_row = "127111"
closest_pair = get_pair_dict(scene_dict[path_row], "closest")
farthest_pair = get_pair_dict(scene_dict[path_row], "farthest")

In [ ]:
path_row_idx = path_rows.index(path_row)
date = dates[path_row_idx][0]

r_url = scene_dict[path_row][date][0][r_channel]
g_url = scene_dict[path_row][date][0][g_channel]
b_url = scene_dict[path_row][date][0][b_channel]

r_aws = scene_dict[path_row][date][0][r_channel + "_alternate"]
g_aws = scene_dict[path_row][date][0][g_channel + "_alternate"]
b_aws = scene_dict[path_row][date][0][b_channel + "_alternate"]

r_band_suffix = os.path.splitext(os.path.basename(r_url))[0].split("_")[-1]
g_band_suffix = os.path.splitext(os.path.basename(g_url))[0].split("_")[-1]
b_band_suffix = os.path.splitext(os.path.basename(b_url))[0].split("_")[-1]

In [ ]:
for pr in path_rows:
    counter = 1
    true_color_dir = f"data/inputs/{dir_suffix}{platform}_{pr}/true_color"
    os.makedirs(true_color_dir, exist_ok=True)

    true_color_ds_dir = f"data/inputs/{dir_suffix}{platform}_{pr}/true_color_ds"
    os.makedirs(true_color_ds_dir, exist_ok=True)

    pr_dict = scene_dict[pr]
    closest_pair = get_pair_dict(pr_dict, "closest")
    farthest_pair = get_pair_dict(pr_dict, "farthest")

    pr_date_list = closest_pair + [farthest_pair[1]]
    for el in pr_date_list:
        print(
            f"Now downloading and processing closest pairs for {el['scene_name']} and path_row: {pr}, scene {counter} from total of 3.",
            end="\r",
        )
        counter += 1
        r_url = el[r_channel + "_alternate"]
        g_url = el[g_channel + "_alternate"]
        b_url = el[b_channel + "_alternate"]
        output_dir = (
            f"data/inputs/{dir_suffix}{platform}_{pr}/Originals/{el['scene_name']}"
        )
        os.makedirs(output_dir, exist_ok=True)
        r_output = os.path.join(output_dir, os.path.basename(r_url))
        g_output = os.path.join(output_dir, os.path.basename(g_url))
        b_output = os.path.join(output_dir, os.path.basename(b_url))
        r_img, r_meta = stream_scene_from_aws(r_url, aws_session)
        g_img, g_meta = stream_scene_from_aws(g_url, aws_session)
        b_img, b_meta = stream_scene_from_aws(b_url, aws_session)

        imgs = [r_img, g_img, b_img]
        outputs = [r_output, g_output, b_output]
        metas = [r_meta, g_meta, b_meta]
        for i, img in enumerate(imgs):
            with rasterio.open(outputs[i], "w", **metas[i]["profile"]) as ds:
                ds.write(img[0, :, :], 1)

        files = glob.glob(f"{output_dir}/**")
        r_band = list(filter(lambda f: f.endswith(f"_{r_band_suffix}.TIF"), files))[0]
        g_band = list(filter(lambda f: f.endswith(f"_{g_band_suffix}.TIF"), files))[0]
        b_band = list(filter(lambda f: f.endswith(f"_{b_band_suffix}.TIF"), files))[0]
        true_bands = [r_band, g_band, b_band]
        tc_file = f"{os.path.join(true_color_dir, os.path.basename(output_dir))}_TC.TIF"
        make_true_color_scene(true_bands, tc_file)
        downsample_dataset(
            tc_file, 0.2, os.path.join(true_color_ds_dir, os.path.basename(tc_file))
        )

        if not keep_original_band_scenes:
            shutil.rmtree(
                f"data/inputs/{dir_suffix}{platform}_{pr}/Originals",
                ignore_errors=True,
            )